# Lab 1 - Overview of embeddings-based retrieval

Welcome! Here's a few notes about the Chroma course notebooks.
 - A number of warnings pop up when running the notebooks. These are normal and can be ignored.
 - Some operations such as calling an LLM or an opeation using generated data return unpredictable results and so your notebook outputs may differ from the video.
  
Enjoy the course!

In [1]:
! pip install chromadb==0.4.18 langchain==0.0.343 openai==1.3.6 pypdf==3.17.1 sentence_transformers==2.2.2

In [2]:
%run notebooks/advanced-retrieval-for-ai-with-chroma/utils.py

In [3]:
from pypdf import PdfReader

PDF_PATH = './notebooks/advanced-retrieval-for-ai-with-chroma'
PDF_NAME = 'microsoft_annual_report_2022.pdf'

reader = PdfReader(f'{PDF_PATH}/{PDF_NAME}')

pdf_texts = [p.extract_text().strip() for p in reader.pages]

pdf_texts = [text for text in pdf_texts if text]

print(word_wrap(pdf_texts[0]))

1 Dear shareholders, colleagues, customers, and partners:  
We are
living through a period of historic economic, societal, and
geopolitical change. The world in 2022 looks nothing like 
the world in
2019. As I write this, inflation is at a 40 -year high, supply chains
are stretched, and the war in Ukraine is 
ongoing. At the same time, we
are entering a technological era with the potential to power awesome
advancements 
across every sector of our economy and society. As the
world’s largest software company, this places us at a historic

intersection of opportunity and responsibility to the world around us.
 
Our mission to empower every person and every organization on the
planet to achieve more has never been more 
urgent or more necessary.
For all the uncertainty in the world, one thing is clear: People and
organizations in every 
industry are increasingly looking to digital
technology to overcome today’s challenges and emerge stronger. And no

company is better positioned to help th

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

character_splitter = RecursiveCharacterTextSplitter(separators=['\n\n', '\n', '. ', ' ', ''],
                                                    chunk_size=1000,
                                                    chunk_overlap=0)

character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(f'Total of chunks: {len(character_split_texts)}')
print(word_wrap(character_split_texts[10]))

Total of chunks: 347
increased, due in large part to significant global datacenter
expansions and the growth in Xbox sales and usage. Despite 
these
increases, we remain dedicated to achieving a net -zero future. We
recognize that progress won’t always be linear, 
and the rate at which
we can implement emissions reductions is dependent on many factors that
can fluctuate over time.  
On the path to becoming water positive, we
invested in 21 water replenishment projects that are expected to
generate 
over 1.3  million cubic meters of volumetric benefits in nine
water basins around the world. Progress toward our zero waste

commitment included diverting more than 15,200 metric tons of solid
waste otherwise headed to landfills and incinerators, 
as well as
launching new Circular Centers to increase reuse and reduce e -waste at
our datacenters.  
We contracted to protect over 17,000 acres of land
(50% more than the land we use to operate), thus achieving our


In [5]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0,
                                                       tokens_per_chunk=256)

token_split_texts = []

for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(f'Total of chunks: {len(token_split_texts)}')
print(word_wrap(token_split_texts[10]))

Total of chunks: 349
increased, due in large part to significant global datacenter
expansions and the growth in xbox sales and usage. despite these
increases, we remain dedicated to achieving a net - zero future. we
recognize that progress won ’ t always be linear, and the rate at which
we can implement emissions reductions is dependent on many factors that
can fluctuate over time. on the path to becoming water positive, we
invested in 21 water replenishment projects that are expected to
generate over 1. 3 million cubic meters of volumetric benefits in nine
water basins around the world. progress toward our zero waste
commitment included diverting more than 15, 200 metric tons of solid
waste otherwise headed to landfills and incinerators, as well as
launching new circular centers to increase reuse and reduce e - waste
at our datacenters. we contracted to protect over 17, 000 acres of land
( 50 % more than the land we use to operate ), thus achieving our


In [6]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

[[0.042562682181596756, 0.033211804926395416, 0.030340079218149185, -0.034866590052843094, 0.06841651350259781, -0.08090908825397491, -0.015474377200007439, -0.0014509431784972548, -0.016744473949074745, 0.06770772486925125, -0.05054134875535965, -0.049195390194654465, 0.05139994993805885, 0.09192727506160736, -0.07177837938070297, 0.03951968625187874, -0.012833536602556705, -0.02494746819138527, -0.04622867330908775, -0.024357527494430542, 0.033949702978134155, 0.025502411648631096, 0.027317123487591743, -0.004126229789108038, -0.036338336765766144, 0.003690892131999135, -0.027430422604084015, 0.004796754103153944, -0.02889624610543251, -0.018870724365115166, 0.036666277796030045, 0.025695830583572388, 0.031312860548496246, -0.06393437087535858, 0.053944025188684464, 0.08225351572036743, -0.04175686836242676, -0.00699581066146493, -0.023486051708459854, -0.03074793517589569, -0.00297919032163918, -0.07790939509868622, 0.009353124536573887, 0.0031628748401999474, -0.022257044911384583,

In [7]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection(PDF_NAME,
                                                    embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids,
                      documents=token_split_texts)

chroma_collection.count()

349

In [8]:
query = 'What was the total revenue?'

results = chroma_collection.query(query_texts=[query],
                                  n_results=5)

retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document), end='\n\n')

revenue, classified by significant product and service offerings, was
as follows : ( in millions ) year ended june 30, 2022 2021 2020 server
products and cloud services $ 67, 321 $ 52, 589 $ 41, 379 office
products and cloud services 44, 862 39, 872 35, 316 windows 24, 761 22,
488 21, 510 gaming 16, 230 15, 370 11, 575 linkedin 13, 816 10, 289 8,
077 search and news advertising 11, 591 9, 267 8, 524 enterprise
services 7, 407 6, 943 6, 409 devices 6, 991 6, 791 6, 457 other 5, 291
4, 479 3, 768 total $ 198, 270 $ 168, 088 $ 143, 015 we have recast
certain previously reported amounts in the table above to conform to
the way we internally manage and monitor our business.

74 note 13 — unearned revenue unearned revenue by segment was as
follows : ( in millions ) june 30, 2022 2021 productivity and business
processes $ 24, 558 $ 22, 120 intelligent cloud 19, 371 17, 710 more
personal computing 4, 479 4, 311 total $ 48, 408 $ 44, 141 changes in
unearned revenue were as follows : ( in millio

In [9]:
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv
from dotenv import find_dotenv

_  = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

In [10]:
def rag(query: str, retrieved_documents: list, model: str='gpt-3.5-turbo') -> str:
    information = '\n\n'.join(retrieved_documents)

    messages = [
        dict(role='system',
             content='You are a helpful expert financial '
                     'research assistant. Your users are '
                     'asking questions about information '
                     'contained in an annual report. '
                     'You will be shown the user\'s question, '
                     'and the relevant information from the '
                     'annual report. Answer the user\'s '
                     'question using only this information.'),
        dict(role='user',
             content=f'Question: {query}. \n Information: {information}')
    ]

    response = openai_client.chat.completions.create(model=model,
                                                     messages=messages)

    content = response.choices[0].message.content

    return content

In [14]:
output = rag(query=query,
             retrieved_documents=retrieved_documents)

print(word_wrap(output))

The total revenue for the year ended June 30, 2022, was $198,270
million.


In [19]:
def ask_me(query: str):
    retrieved_documents = chroma_collection.query(query_texts=[query],
                                                  n_results=5)
    
    return rag(query=query,
               retrieved_documents=retrieved_documents)

In [20]:
ask_me('How COVID-19 and Ukranie war impact the Microsoft\'s culture?')

"Based on the given information, there is no direct information about how COVID-19 and the Ukraine war have specifically impacted Microsoft's culture. The information provided (ids, distances, metadatas, embeddings, documents, uris, data) does not provide any insight into the effects of these events on Microsoft's culture."

In [21]:
ask_me('How Microsoft handled diversity and inclusion?')

"I'm sorry, but the information provided does not pertain to Microsoft's handling of diversity and inclusion. Could you please provide more specific information or rephrase your question?"

In [22]:
ask_me('What improvements Microsoft did in order to create new ways of working?')

'Based on the information provided in the annual report, it is unclear what specific improvements Microsoft made in order to create new ways of working. The information provided includes terms such as ids, distances, metadatas, embeddings, documents, uris, and data, but there is no direct mention of the improvements made by Microsoft. It would be helpful to have more context or specific details to provide a more accurate answer.'

In [23]:
ask_me('What is the total of employees on full time basis?')

"I'm sorry, but based on the information provided, there is no mention of the number of employees or any information related to full-time basis in the provided data. Please provide additional information or refer to a specific section of the annual report for further details."

In [24]:
ask_me('How many people was employed, in 2022, in product research and development?')

"I'm sorry, but the information provided does not include any details about the number of people employed in product research and development in 2022."